# First tests

## Install Dependencies

In [ ]:
# ! is used to run console commands in jupyter notebooks
!pip install -q nbstripout
#!pip install LIBRARYNAME

## Import Dependencies

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

## Set Variables

In [ ]:
path = './KaggleCache/datasets/andrewmvd/ocular-disease-recognition-odir5k/versions/2'
df = pd.read_csv(os.path.join(path, 'full_df.csv'))

## TestModel

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# ==========================================
# 1. KONFIGURATION & PFADE
# ==========================================
# Pfad zu den Bildern (wie von dir angegeben)
IMG_DIR = "./KaggleCache/datasets/andrewmvd/ocular-disease-recognition-odir5k/versions/2/preprocessed_images"
# Pfad zur CSV (Annahme: liegt im versions/2 Ordner)
CSV_PATH = "./KaggleCache/datasets/andrewmvd/ocular-disease-recognition-odir5k/versions/2/full_df.csv"

# Hyperparameter
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_EPOCHS = 10       # Für erste Ergebnisse reichen 10, später auf 30-50 erhöhen
IMG_SIZE = 224        # Resize auf 224x224 für das CNN
NUM_CLASSES = 8
CLASS_NAMES = ['Normal', 'Diabetes', 'Glaucoma', 'Cataract', 'AMD', 'Hypertension', 'Myopia', 'Other']

# Hardware Check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training läuft auf: {device}")

# ==========================================
# 2. DATEN VORBEREITUNG (WRANGLING)
# ==========================================
def prepare_data(csv_path):
    print("Lade und verarbeite Metadaten...")
    df = pd.read_csv(csv_path)
    
    # Mapping der ODIR-Kürzel auf Indizes
    # N, D, G, C, A, H, M, O
    target_cols = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']
    
    # Aufteilen in Linkes und Rechtes Auge (Explosion des Datasets)
    left_df = df[['ID', 'Left-Fundus'] + target_cols].copy()
    left_df.rename(columns={'Left-Fundus': 'filename'}, inplace=True)
    
    right_df = df[['ID', 'Right-Fundus'] + target_cols].copy()
    right_df.rename(columns={'Right-Fundus': 'filename'}, inplace=True)
    
    combined_df = pd.concat([left_df, right_df], axis=0)
    
    # Erstelle Labels (0-7) aus One-Hot Encoding
    labels = combined_df[target_cols].values
    combined_df['label'] = np.argmax(labels, axis=1)
    
    # Optional: Prüfen ob Datei existiert (kann man auskommentieren für Speed)
    # combined_df['file_path'] = combined_df['filename'].apply(lambda x: os.path.join(IMG_DIR, x))
    
    return combined_df

full_df = prepare_data(CSV_PATH)

# ==========================================
# 3. SPLIT (PATIENTEN-BASIERT) 70/15/15
# ==========================================
# Wichtig: Wir splitten IDs, nicht Bilder, um Data Leakage zu verhindern!
patient_ids = full_df['ID'].unique()
train_ids, test_ids = train_test_split(patient_ids, test_size=0.3, random_state=42)
val_ids, test_ids = train_test_split(test_ids, test_size=0.5, random_state=42)

train_df = full_df[full_df['ID'].isin(train_ids)].reset_index(drop=True)
val_df = full_df[full_df['ID'].isin(val_ids)].reset_index(drop=True)
test_df = full_df[full_df['ID'].isin(test_ids)].reset_index(drop=True)

print(f"   Datensatz Split:")
print(f"   Training:   {len(train_df)} Bilder")
print(f"   Validierung:{len(val_df)} Bilder")
print(f"   Test:       {len(test_df)} Bilder")

# ==========================================
# 4. KLASSENGEWICHTE BERECHNEN
# ==========================================
# Da der Datensatz extrem unbalanciert ist (viele N/D, wenig A/G etc.)
print("Berechne Class Weights...")
y_train = train_df['label'].values
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
print(f"   Gewichte: {class_weights.cpu().numpy()}")

# ==========================================
# 5. DATASET CLASS & DATALOADER
# ==========================================
class ODIRDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.df = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.root_dir, row['filename'])
        label = row['label']
        
        try:
            # Bild laden
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            # Fallback für korrupte Bilder (schwarzes Bild)
            # print(f"Warning: Could not load {img_path}")
            image = Image.new('RGB', (IMG_SIZE, IMG_SIZE))
            
        if self.transform:
            image = self.transform(image)
            
        return image, torch.tensor(label, dtype=torch.long)

# Transformationen (Augmentation für Train, Resize für alle)
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Loader erstellen
train_dataset = ODIRDataset(train_df, IMG_DIR, transform=train_transforms)
val_dataset = ODIRDataset(val_df, IMG_DIR, transform=val_test_transforms)
test_dataset = ODIRDataset(test_df, IMG_DIR, transform=val_test_transforms)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# ==========================================
# 6. CUSTOM CNN ARCHITEKTUR (VGG-Style)
# ==========================================
class ODIR_CNN(nn.Module):
    def __init__(self, num_classes=8):
        super(ODIR_CNN, self).__init__()
        
        # Block 1
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2) # Output: 112x112
        )
        # Block 2
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2) # Output: 56x56
        )
        # Block 3
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2) # Output: 28x28
        )
        # Block 4
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2) # Output: 14x14
        )
        
        # Flatten Dimension: 256 Channels * 14 * 14 Pixel
        self.flatten_dim = 256 * 14 * 14 
        
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.flatten_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.fc(x)
        return x

# Modell initialisieren
model = ODIR_CNN(num_classes=NUM_CLASSES).to(device)

# Loss Function & Optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# ==========================================
# 7. TRAINING LOOP
# ==========================================
train_losses, val_losses = [], []
train_accs, val_accs = [], []

print(f"Starte Training für {NUM_EPOCHS} Epochen...")

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    # Training
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct_train / total_train
    train_losses.append(epoch_loss)
    train_accs.append(epoch_acc)
    
    # Validation
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
            
    epoch_val_loss = val_loss / len(val_loader)
    epoch_val_acc = 100 * correct_val / total_val
    val_losses.append(epoch_val_loss)
    val_accs.append(epoch_val_acc)
    
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | "
          f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.2f}% | "
          f"Val Loss: {epoch_val_loss:.4f} Acc: {epoch_val_acc:.2f}%")

print("Training beendet.")

# ==========================================
# 8. EVALUATION & VISUALISIERUNG
# ==========================================
# Plotten der Lernkurven
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.title('Loss über Epochen')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train Acc')
plt.plot(val_accs, label='Val Acc')
plt.title('Accuracy über Epochen')
plt.legend()
plt.show()

# Finaler Test & Confusion Matrix
print("Generiere Confusion Matrix auf Testdaten...")
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.numpy())

# Report
print(classification_report(all_labels, all_preds, target_names=CLASS_NAMES, zero_division=0))

# Matrix Plot
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import torch
#import torchvision
print(f"Torch Version: {torch.__version__}")
#print(f"Torchvision Version: {torchvision.__version__}")

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import cv2
import time
from torchvision import models
import albumentations as A
from albumentations.pytorch import ToTensorV2

# ==========================================
# VERBESSERTE KONFIGURATION
# ==========================================
CONFIG = {
    "Project": "ODIR-5K Transfer Learning",
    "Img_Dir": "./KaggleCache/datasets/andrewmvd/ocular-disease-recognition-odir5k/versions/2/preprocessed_images",
    "CSV_Path": "./KaggleCache/datasets/andrewmvd/ocular-disease-recognition-odir5k/versions/2/full_df.csv",
    "Batch_Size": 16,  # Kleiner für stabileres Training
    "Learning_Rate": 1e-4,  # Konservativer Start
    "Epochs": 30,
    "Img_Size": 224,
    "Num_Classes": 8,
    "Architecture": "EfficientNet-B3 (Pretrained)",
    "Device": "cuda" if torch.cuda.is_available() else "cpu",
    "Mixed_Precision": True,  # Schnelleres Training mit AMP
    "Early_Stop_Patience": 7,
    "Gradient_Clip": 1.0
}

def print_config(conf):
    print("\n" + "="*50)
    print(f"🔬 ODIR-5K TRAINING PIPELINE")
    print("="*50)
    for k, v in conf.items():
        print(f"{k:<20}: {v}")
    print("="*50 + "\n")

print_config(CONFIG)
device = torch.device(CONFIG["Device"])

# ==========================================
# DATA PREPARATION
# ==========================================
def prepare_data(csv_path, img_dir):
    df = pd.read_csv(csv_path)
    target_cols = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']
    
    left_df = df[['ID', 'Left-Fundus'] + target_cols].copy()
    left_df['filename'] = left_df['Left-Fundus']
    left_df = left_df.drop('Left-Fundus', axis=1)
    
    right_df = df[['ID', 'Right-Fundus'] + target_cols].copy()
    right_df['filename'] = right_df['Right-Fundus']
    right_df = right_df.drop('Right-Fundus', axis=1)
    
    combined_df = pd.concat([left_df, right_df], axis=0, ignore_index=True)
    combined_df['full_path'] = combined_df['filename'].apply(lambda x: os.path.join(img_dir, x))
    
    # Nur existierende Bilder
    combined_df = combined_df[combined_df['full_path'].map(os.path.exists)].reset_index(drop=True)
    combined_df['label'] = np.argmax(combined_df[target_cols].values, axis=1)
    
    print(f"📊 Datensatz geladen: {len(combined_df)} Bilder")
    print(f"Klassen-Verteilung:\n{combined_df['label'].value_counts().sort_index()}\n")
    
    return combined_df, target_cols

full_df, target_cols = prepare_data(CONFIG["CSV_Path"], CONFIG["Img_Dir"])

# Patient-Level Split (wichtig!)
patient_ids = full_df['ID'].unique()
train_ids, temp_ids = train_test_split(patient_ids, test_size=0.3, random_state=42, stratify=None)
val_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)

train_df = full_df[full_df['ID'].isin(train_ids)].reset_index(drop=True)
val_df = full_df[full_df['ID'].isin(val_ids)].reset_index(drop=True)
test_df = full_df[full_df['ID'].isin(test_ids)].reset_index(drop=True)

print(f"Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

# Class Weights
y_train = train_df['label'].values
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
print(f"Class Weights: {class_weights.cpu().numpy()}\n")

# ==========================================
# MODERNE AUGMENTATION (Albumentations)
# ==========================================
train_transform = A.Compose([
    A.Resize(CONFIG["Img_Size"], CONFIG["Img_Size"]),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.3),
    A.RandomRotate90(p=0.3),
    A.Affine(scale=(0.9, 1.1), translate_percent=0.1, rotate=(-15, 15), p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.GaussNoise(p=0.3),  # Simplified without var_limit
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CONFIG["Img_Size"], CONFIG["Img_Size"]),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# ==========================================
# DATASET
# ==========================================
class ODIRDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.iloc[idx]['full_path']
        label = self.df.iloc[idx]['label']
        
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        return image, torch.tensor(label, dtype=torch.long)

train_dataset = ODIRDataset(train_df, transform=train_transform)
val_dataset = ODIRDataset(val_df, transform=val_transform)
test_dataset = ODIRDataset(test_df, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=CONFIG["Batch_Size"], 
                          shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG["Batch_Size"], 
                        shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=CONFIG["Batch_Size"], 
                         shuffle=False, num_workers=4, pin_memory=True)

# ==========================================
# TRANSFER LEARNING MODEL (EfficientNet)
# ==========================================
class ODIRModel(nn.Module):
    def __init__(self, num_classes=8, pretrained=True):
        super(ODIRModel, self).__init__()
        
        # EfficientNet-B3 als Backbone
        self.backbone = models.efficientnet_b3(weights='IMAGENET1K_V1' if pretrained else None)
        
        # Anzahl Features aus dem letzten Layer
        num_ftrs = self.backbone.classifier[1].in_features
        
        # Custom Classifier Head
        self.backbone.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        return self.backbone(x)

model = ODIRModel(num_classes=CONFIG["Num_Classes"]).to(device)

# Optimizer mit unterschiedlichen LRs für Backbone vs. Head
optimizer = optim.AdamW([
    {'params': model.backbone.features.parameters(), 'lr': CONFIG["Learning_Rate"] * 0.1},  # Backbone langsamer
    {'params': model.backbone.classifier.parameters(), 'lr': CONFIG["Learning_Rate"]}  # Head schneller
], weight_decay=0.01)

criterion = nn.CrossEntropyLoss(weight=class_weights)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
scaler = torch.amp.GradScaler('cuda', enabled=CONFIG["Mixed_Precision"])

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"🧠 Modell: EfficientNet-B3")
print(f"   Total Parameters: {total_params:,}")
print(f"   Trainable: {trainable_params:,}\n")

# ==========================================
# TRAINING LOOP MIT EARLY STOPPING
# ==========================================
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    avg_loss = total_loss / len(loader)
    accuracy = 100 * np.mean(np.array(all_preds) == np.array(all_labels))
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    return avg_loss, accuracy, f1, all_preds, all_labels

print(f"🚀 Starte Training ({CONFIG['Epochs']} Epochen max)...\n")
start_time = time.time()

best_val_f1 = 0.0
patience_counter = 0
train_losses, val_losses = [], []

for epoch in range(CONFIG["Epochs"]):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        # Mixed Precision Training
        with torch.amp.autocast('cuda', enabled=CONFIG["Mixed_Precision"]):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), CONFIG["Gradient_Clip"])
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item()
    
    avg_train_loss = running_loss / len(train_loader)
    val_loss, val_acc, val_f1, _, _ = evaluate(model, val_loader, criterion, device)
    
    train_losses.append(avg_train_loss)
    val_losses.append(val_loss)
    
    print(f"Epoch {epoch+1:02d}/{CONFIG['Epochs']} | "
          f"Train Loss: {avg_train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | "
          f"Val Acc: {val_acc:.2f}% | "
          f"Val F1: {val_f1:.4f}")
    
    scheduler.step()
    
    # Early Stopping Check
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"   ✅ Neues Best Model (F1: {val_f1:.4f})")
    else:
        patience_counter += 1
        if patience_counter >= CONFIG["Early_Stop_Patience"]:
            print(f"\n⏹️  Early Stopping nach {epoch+1} Epochen")
            break

duration = (time.time() - start_time) / 60
print(f"\n✅ Training beendet in {duration:.1f} Minuten")

# ==========================================
# FINAL EVALUATION
# ==========================================
print("\n" + "="*50)
print("📈 FINALE EVALUATION AUF TEST SET")
print("="*50)

model.load_state_dict(torch.load('best_model.pth'))
test_loss, test_acc, test_f1, test_preds, test_labels = evaluate(model, test_loader, criterion, device)

print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.2f}%")
print(f"Test F1-Score: {test_f1:.4f}\n")

print("Detaillierter Classification Report:")
print(classification_report(test_labels, test_preds, target_names=target_cols, digits=3))

print("\nConfusion Matrix:")
cm = confusion_matrix(test_labels, test_preds)
print(cm)

In [ ]:
!pip install albumentations --quiet

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# ==========================================
# FIX #1: FOCAL LOSS (statt CrossEntropyLoss)
# ==========================================
# Fokussiert sich auf schwierige Beispiele
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha  # Class weights
        self.gamma = gamma  # Focusing parameter
        self.reduction = reduction
    
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, weight=self.alpha, reduction='none')
        pt = torch.exp(-ce_loss)  # Probability of correct class
        focal_loss = (1 - pt) ** self.gamma * ce_loss
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# ==========================================
# FIX #2: LABEL SMOOTHING
# ==========================================
# Verhindert Overconfidence (zu sichere Vorhersagen)
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, epsilon=0.1, weight=None):
        super().__init__()
        self.epsilon = epsilon
        self.weight = weight
    
    def forward(self, preds, targets):
        n_classes = preds.size(-1)
        log_preds = F.log_softmax(preds, dim=-1)
        
        # Smooth labels
        with torch.no_grad():
            true_dist = torch.zeros_like(log_preds)
            true_dist.fill_(self.epsilon / (n_classes - 1))
            true_dist.scatter_(1, targets.unsqueeze(1), 1.0 - self.epsilon)
        
        if self.weight is not None:
            loss = -torch.sum(true_dist * log_preds, dim=-1)
            loss = loss * self.weight[targets]
            return loss.mean()
        
        return torch.mean(-torch.sum(true_dist * log_preds, dim=-1))

# ==========================================
# FIX #3: TEST TIME AUGMENTATION (TTA)
# ==========================================
def tta_predict(model, dataloader, device, n_tta=3):
    """
    Test Time Augmentation - macht mehrere Vorhersagen pro Bild
    und mittelt diese für stabilere Predictions
    """
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Original prediction
            outputs = model(inputs)
            
            # TTA: Horizontal Flip
            outputs_hflip = model(torch.flip(inputs, dims=[3]))
            
            # TTA: Vertical Flip (optional)
            outputs_vflip = model(torch.flip(inputs, dims=[2]))
            
            # Average predictions
            outputs_avg = (outputs + outputs_hflip + outputs_vflip) / 3.0
            
            _, preds = torch.max(outputs_avg, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    return np.array(all_preds), np.array(all_labels)

# ==========================================
# KOMPLETTES RE-TRAINING MIT FIXES
# ==========================================
print("🔥 STARTE VERBESSERTES TRAINING")
print("="*60)

# Model neu initialisieren
model_improved = models.efficientnet_b3(weights='IMAGENET1K_V1')
num_ftrs = model_improved.classifier[1].in_features
model_improved.classifier = nn.Sequential(
    nn.Dropout(0.4),  # Etwas mehr Dropout
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, CONFIG["Num_Classes"])
)
model_improved = model_improved.to(device)

# WÄHLE LOSS FUNCTION:
# Option A: Focal Loss (beste für Imbalance)
criterion_improved = FocalLoss(alpha=class_weights, gamma=2.0)

# Option B: Label Smoothing (verhindert Overconfidence)
# criterion_improved = LabelSmoothingCrossEntropy(epsilon=0.1, weight=class_weights)

# Optimizer mit höherer Weight Decay
optimizer_improved = torch.optim.AdamW([
    {'params': model_improved.features.parameters(), 'lr': CONFIG["Learning_Rate"] * 0.1},
    {'params': model_improved.classifier.parameters(), 'lr': CONFIG["Learning_Rate"]}
], weight_decay=0.02)  # Mehr Regularization

scheduler_improved = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer_improved, T_0=10, T_mult=2
)
scaler = torch.amp.GradScaler('cuda', enabled=CONFIG["Mixed_Precision"])

# ==========================================
# TRAINING LOOP (kompakt)
# ==========================================
print(f"✅ Loss: Focal Loss (gamma=2.0)")
print(f"✅ Weight Decay: 0.02 (mehr Regularization)")
print(f"✅ Dropout: 0.4/0.3 (gegen Overfitting)")
print("="*60 + "\n")

best_f1_improved = 0.0
patience = 0
epochs_to_train = 25  # Etwas länger

for epoch in range(epochs_to_train):
    # Training
    model_improved.train()
    train_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer_improved.zero_grad()
        
        with torch.amp.autocast('cuda', enabled=CONFIG["Mixed_Precision"]):
            outputs = model_improved(inputs)
            loss = criterion_improved(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer_improved)
        torch.nn.utils.clip_grad_norm_(model_improved.parameters(), 1.0)
        scaler.step(optimizer_improved)
        scaler.update()
        train_loss += loss.item()
    
    # Validation
    model_improved.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_improved(inputs)
            _, preds = torch.max(outputs, 1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
    
    val_acc = 100 * np.mean(np.array(val_preds) == np.array(val_labels))
    val_f1 = f1_score(val_labels, val_preds, average='weighted')
    
    print(f"Epoch {epoch+1:02d}/{epochs_to_train} | "
          f"Train Loss: {train_loss/len(train_loader):.4f} | "
          f"Val Acc: {val_acc:.2f}% | Val F1: {val_f1:.4f}")
    
    scheduler_improved.step()
    
    # Early stopping
    if val_f1 > best_f1_improved:
        best_f1_improved = val_f1
        torch.save(model_improved.state_dict(), 'best_model_improved.pth')
        print(f"   ✅ New Best (F1: {val_f1:.4f})")
        patience = 0
    else:
        patience += 1
        if patience >= 7:
            print(f"\n⏹️  Early Stop at Epoch {epoch+1}")
            break

# ==========================================
# EVALUATION MIT TTA
# ==========================================
print("\n" + "="*60)
print("📊 EVALUATION: STANDARD vs. TTA")
print("="*60)

model_improved.load_state_dict(torch.load('best_model_improved.pth', weights_only=True))

# Standard Evaluation
model_improved.eval()
test_preds_std, test_labels_std = [], []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_improved(inputs)
        _, preds = torch.max(outputs, 1)
        test_preds_std.extend(preds.cpu().numpy())
        test_labels_std.extend(labels.cpu().numpy())

std_acc = 100 * np.mean(np.array(test_preds_std) == np.array(test_labels_std))
std_f1 = f1_score(test_labels_std, test_preds_std, average='weighted')

print(f"\n📌 STANDARD PREDICTION:")
print(f"   Accuracy: {std_acc:.2f}%")
print(f"   F1-Score: {std_f1:.4f}")

# TTA Evaluation
print(f"\n📌 TTA PREDICTION (3x Augmentation):")
test_preds_tta, test_labels_tta = tta_predict(model_improved, test_loader, device, n_tta=3)

tta_acc = 100 * np.mean(test_preds_tta == test_labels_tta)
tta_f1 = f1_score(test_labels_tta, test_preds_tta, average='weighted')

print(f"   Accuracy: {tta_acc:.2f}% (+{tta_acc-std_acc:.2f}%)")
print(f"   F1-Score: {tta_f1:.4f} (+{tta_f1-std_f1:.4f})")

print("\n" + "="*60)
print("📋 DETAILED REPORT (TTA):")
print("="*60)
print(classification_report(test_labels_tta, test_preds_tta, 
                          target_names=target_cols, digits=3))

print("\n🎯 CONFUSION MATRIX (TTA):")
cm = confusion_matrix(test_labels_tta, test_preds_tta)
print(cm)

print("\n" + "="*60)
print("💾 Models saved:")
print("   - best_model_improved.pth")
print("="*60)